### Imports

In [1]:
from __future__ import division, print_function
from collections import OrderedDict

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt
import time
import os
import cv2
from PIL import Image, ImageChops

plt.ion()   # interactive mode
%matplotlib inline
%env TMP="/tmp"
%env JOBLIB_TEMP_FOLDER="/tmp"

env: TMP="/tmp"
env: JOBLIB_TEMP_FOLDER="/tmp"


In [2]:
def summary(input_size, model):
    def register_hook(module):
        def hook(module, input, output):
            class_name = str(module.__class__).split('.')[-1].split("'")[0]
            module_idx = len(summary)

            m_key = '%s-%i' % (class_name, module_idx+1)
            summary[m_key] = OrderedDict()
            summary[m_key]['input_shape'] = list(input[0].size())
            summary[m_key]['input_shape'][0] = -1
            summary[m_key]['output_shape'] = list(output.size())
            summary[m_key]['output_shape'][0] = -1

            params = 0
            if hasattr(module, 'weight'):
                params += torch.prod(torch.LongTensor(list(module.weight.size())))
                if module.weight.requires_grad:
                    summary[m_key]['trainable'] = True
                else:
                    summary[m_key]['trainable'] = False
            if hasattr(module, 'bias'):
                params +=  torch.prod(torch.LongTensor(list(module.bias.size())))
            summary[m_key]['nb_params'] = params

        if not isinstance(module, nn.Sequential) and \
           not isinstance(module, nn.ModuleList) and \
           not (module == model):
            hooks.append(module.register_forward_hook(hook))

    # check if there are multiple inputs to the network
    if isinstance(input_size[0], (list, tuple)):
        x = [Variable(torch.rand(1,*in_size)) for in_size in input_size]
    else:
        x = Variable(torch.rand(1,*input_size))

    # create properties
    summary = OrderedDict()
    hooks = []
    # register hook
    model.apply(register_hook)
    # make a forward pass
    model(x)
    # remove these hooks
    for h in hooks:
        h.remove()

    return summary

### Data

In [3]:
TRAIN_DIR = '../data/boneage-training-dataset/'
TRAIN_LABEL = '../data/train.csv'
FILETYPE = '.png'
use_gpu = torch.cuda.is_available()

### Dataset class

In [4]:
class BoneAgeDataset(Dataset):
    """Bone Age dataset."""

    def __init__(self, root_dir, labels, filetype, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
        self.filetype = filetype

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.labels.iloc[idx]['id']) + self.filetype)
        img = Image.open(img_name).convert('RGB')
        img = img.point(lambda x: x if x<200 else 0)
#         print('PIL size:', img.size)
        img = np.asarray(img)
#         print('Np size: ',img.shape)
        if img.shape[1] > img.shape[0]:
#             print(img.shape)
            img = np.swapaxes(img,0,1)
        img = cv2.resize(img, (640,850))
        img = img[210:,:]
#         print(img.shape)
        rot_ang = range(0,360,30)
        rot_ang = rot_ang[np.random.randint(0,len(rot_ang))]
#         print(rot_ang)
        scale = np.random.uniform(0.8,1.2)
#         print(scale)
        rot_mat = cv2.getRotationMatrix2D((img.shape[0]/2,img.shape[1]/2), rot_ang, scale);
        img = cv2.warpAffine(img, rot_mat, (img.shape[0],img.shape[1]))
#         print(img.shape)
        img = Image.fromarray(np.uint8(img))
        
#         image = cv2.imread(img_name)[:,:,::-1]
        boneage = self.labels.iloc[idx]['boneage'].astype(np.float32)
        gender = self.labels.iloc[idx]['male'].astype(np.uint8)
        
        label = np.hstack((boneage, gender))

        if self.transform:
            img = self.transform(img)
            
        sample = {'image': img, 'label': label}

        return sample

### Transforms

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale(320),
#         transforms.CenterCrop(512),
        transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Scale(320),
#         transforms.CenterCrop(512),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

### Data loaders

In [6]:
labels = pd.read_csv(TRAIN_LABEL)
bins = xrange(20)
labels['bins'] = np.digitize(labels['boneage'], bins=bins)

In [7]:
skf = StratifiedKFold(n_splits=4)
splits = []
for train, test in skf.split(np.zeros(len(labels)), labels['bins']):
    fold_label = {
        'train': labels.loc[train, ('id', 'boneage', 'male')],
        'val': labels.loc[test, ('id', 'boneage', 'male')]
    }
    splits.append(fold_label)

/home/user/software/miniconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


In [8]:
fold_label = splits[0]

In [9]:
image_datasets = {x: BoneAgeDataset(TRAIN_DIR, fold_label[x], FILETYPE, data_transforms[x])
                  for x in ['train', 'val']}
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=16, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [10]:
for i_batch, sample in enumerate(dataloders['val']):
    print(i_batch, sample['image'].size())
    print(i_batch, sample['image'].mean())
    if i_batch == 2:
        break

0 torch.Size([16, 3, 224, 224])
0 -0.904783348563
1 torch.Size([16, 3, 224, 224])
1 -1.2249584799
2 torch.Size([16, 3, 224, 224])
2 -0.761572569647


### Training

In [16]:
class Net(nn.Module):
    def __init__(self, input_shape=(3, 224, 224)):
        super(Net, self).__init__()
        
        self.bn = nn.BatchNorm2d(input_shape[0])
        resnet = models.resnet18(pretrained=False)
        num_ftrs = resnet.fc.in_features
        resnet.fc = nn.Linear(num_ftrs, 2)

        self.resnet = resnet

    def forward(self, x):
        x = self.resnet(self.bn(x))
        return x
    
model = Net([3, 224, 224])
    
# print(summary([3, 224, 224], model))

if use_gpu:
    model.cuda()

In [17]:
lr = 0.01
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [18]:
def ccc(x, y):
#     print('X:', x)
    mean_x = torch.mean(x)
#     print('Mean x:', mean_x)
    mean_y = torch.mean(y)
#     print('Mean y:', mean_y)
    xm = x.sub(mean_x)
    ym = y.sub(mean_y)
    xn = torch.norm(xm, 2)
#     print('Xn:', xn)
    yn = torch.norm(ym, 2)
#     print('Yn:', yn)
    xydot = ym.dot(xm.view(-1))
#     print('XYdot:', xydot)

    ccc_num = 2 * xydot / xm.size()[0]
#     print('CCC num:', ccc_num)
    ccc_den = xn + yn + torch.pow(mean_x.sub(mean_y), 2)
#     print('CCC den:', ccc_den)

    ccc = ccc_num / ccc_den
    return ccc
    

def train(epoch, train_loader):
    exp_lr_scheduler.step()
    model.train()
    train_boneage_loss = 0
    train_gender_loss = 0
    train_loss = 0
    train_ccc = 0
    for batch_idx, sample in enumerate(train_loader):
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        
        for i in xrange(output.size[0]):
            if output[i,1] > 0.5:
                output[i,0] = (output[i,0] // 6) * 6
            else:
                output[i,0] = (output[i,0] // 2) * 2
        
        boneage_loss = F.l1_loss(output[:,0], target[:,0])
        train_boneage_loss += boneage_loss.data[0]
        gender_loss = F.binary_cross_entropy(F.sigmoid(output[:,1]), target[:,1])
        train_gender_loss += gender_loss.data[0]
        loss = 0.8 * boneage_loss + 0.2 * gender_loss
        train_loss += loss.data[0]
        c = ccc(output[:,0], target[:,0])
        train_ccc += c.data.cpu().numpy()[0]
        loss.backward()
        optimizer.step()
#         if batch_idx % (1000 // len(target)) == 0:
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tBoneAge Loss: {:.4f}\tGender Loss: {:.4f}\t Loss: {:.4f}\tCCC: {:.4f}'.format(
        epoch, batch_idx * len(data), len(train_loader),
        100. * batch_idx / len(train_loader), train_boneage_loss / len(train_loader),
        train_gender_loss / len(train_loader), train_loss / len(train_loader),
        train_ccc / len(train_loader)))

def test(test_loader):
    model.eval()
    test_boneage_loss = 0
    test_gender_loss = 0
    test_loss = 0
    test_ccc = 0
    for sample in test_loader:
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        boneage_loss = F.l1_loss(output[:,0], target[:,0])
        test_boneage_loss += boneage_loss.data[0]
        gender_loss = F.binary_cross_entropy(F.sigmoid(output[:,1]), target[:,1])
        test_gender_loss += gender_loss.data[0]
        loss = 0.8 * boneage_loss + 0.2 * gender_loss
        test_loss += loss.data[0]
        c = ccc(output[:,0], target[:,0])
        test_ccc += c.data.cpu().numpy()[0]

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Avg BoneAge loss: {:.4f}, Avg Gender loss: {:.4f}, Avg loss: {:.4f} CCC: {:.4f}\n'.format(
        test_boneage_loss / len(test_loader), test_gender_loss / len(test_loader),
        test_loss / len(test_loader), test_ccc / len(test_loader)))

In [19]:
# torch.save(model.state_dict(), '../models/9-27-1540-resnet50-1fold-5.7889-2.56.pth')

In [ ]:
for epoch in range(1, 70 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

Train Epoch: 1 [7670/591 (100%)]	BoneAge Loss: 33.7842	Gender Loss: 0.8518	 Loss: 68.4201	CCC: 0.1757

Test set: Avg BoneAge loss: 38.4368, Avg Gender loss: 1.0430, Avg loss: 0.0247 CCC: -0.1308

Train Epoch: 2 [7670/591 (100%)]	BoneAge Loss: 31.8530	Gender Loss: 0.8281	 Loss: 64.5340	CCC: 0.5336

Test set: Avg BoneAge loss: 38.8241, Avg Gender loss: 0.9699, Avg loss: 0.0249 CCC: 0.0745

Train Epoch: 3 [7670/591 (100%)]	BoneAge Loss: 31.3127	Gender Loss: 0.8128	 Loss: 63.4383	CCC: 0.7426

Test set: Avg BoneAge loss: 42.1918, Avg Gender loss: 0.7551, Avg loss: 0.0270 CCC: -0.4734

Train Epoch: 4 [7670/591 (100%)]	BoneAge Loss: 31.1219	Gender Loss: 0.7900	 Loss: 63.0339	CCC: 0.8862

Test set: Avg BoneAge loss: 44.8073, Avg Gender loss: 0.7096, Avg loss: 0.0286 CCC: -0.2974

Train Epoch: 5 [7670/591 (100%)]	BoneAge Loss: 30.3794	Gender Loss: 0.7693	 Loss: 61.5282	CCC: 1.1378

Test set: Avg BoneAge loss: 49.2186, Avg Gender loss: 0.6990, Avg loss: 0.0314 CCC: -0.3985

Train Epoch: 6 [7670/

Train Epoch: 43 [7670/591 (100%)]	BoneAge Loss: 22.3466	Gender Loss: 0.6869	 Loss: 45.3800	CCC: 4.4852

Test set: Avg BoneAge loss: 37.2269, Avg Gender loss: 0.7150, Avg loss: 0.0238 CCC: 1.6468

Train Epoch: 44 [7670/591 (100%)]	BoneAge Loss: 22.1754	Gender Loss: 0.6884	 Loss: 45.0393	CCC: 4.4603

Test set: Avg BoneAge loss: 39.6905, Avg Gender loss: 0.7145, Avg loss: 0.0254 CCC: 1.4853

Train Epoch: 45 [7670/591 (100%)]	BoneAge Loss: 22.0891	Gender Loss: 0.6873	 Loss: 44.8654	CCC: 4.5061

Test set: Avg BoneAge loss: 37.6796, Avg Gender loss: 0.7135, Avg loss: 0.0241 CCC: 1.7270

Train Epoch: 46 [7670/591 (100%)]	BoneAge Loss: 22.0407	Gender Loss: 0.6875	 Loss: 44.7689	CCC: 4.5500

Test set: Avg BoneAge loss: 39.1904, Avg Gender loss: 0.7326, Avg loss: 0.0251 CCC: 1.7811

Train Epoch: 47 [7670/591 (100%)]	BoneAge Loss: 22.3725	Gender Loss: 0.6874	 Loss: 45.4325	CCC: 4.4339

Test set: Avg BoneAge loss: 37.2462, Avg Gender loss: 0.7182, Avg loss: 0.0238 CCC: 1.8752

Train Epoch: 48 [767